In [8]:
# Import libraries
import cv2
from keras.models import load_model
import numpy as np
from pygame import mixer
import pyttsx3
import time

# detect when audio sound stops
sound_play = 0
prev_detect = ''
last_played = ''
    
# Initialise mixer
mixer.init()
# Load the model
model = load_model('model.h5')
# Create a list containing the trained labels 
model_dict = []
for line in open('labels.txt','r').readlines():
    model_dict.append(line.strip())
# function to play sound
def play_sound(filename):
    # stop any existing sound
    if mixer.music.get_busy():
        mixer.music.pause()
        mixer.music.set_pos(0)
        mixer.music.stop()
    # Play Sound new file
    mixer.music.load(filename)
    if not mixer.music.get_busy():  # wait for music to finish playing
        mixer.music.play()
    # return back
    return
    
# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
# read camera video feed
while cap.isOpened():
    global sound_play
    # read webcam
    ret, frame = cap.read()
    # Cut down frame to 250x250px
    frame = frame[120:120 + 250, 200:200 + 250, :]
    # resize video frame
    frame = cv2.resize(frame, (600, 500))
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    # Get the number of detected face values
    num_faces: int = int(len(faces))
    # Monitor inactive face capture session
    if num_faces <= 0:
        # stop sound
        if mixer.music.get_busy():
            mixer.music.pause()
            mixer.music.set_pos(0)
            mixer.music.stop()
    
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        # draw the rectangle on the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        # gray value
        gray_frame = gray[y:y + h, x:x + w]
        # crop the face image to 48 x 48 pixel 
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(gray_frame, (48, 48)), -1), 0)
        # predict facial expression
        prediction = model.predict(cropped_img)
        # get the maximum predicted index
        maxindex = int(np.argmax(prediction))
        # get the expression from the dictionary
        condition = str(model_dict[maxindex])
        # Blue color in BGR
        cv2.putText(frame, model_dict[maxindex] + ' face. ' + str(sound_play) + '% verified!',
                    (x-60, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        # wait for music to finish playing
        if mixer.music.get_busy() and last_played.find(condition) != -1:  
            sound_play = 0
            
        # already detected
        if last_played.find(condition) != -1:
            sound_play = 100
            
        # check if string exists
        if prev_detect.find(condition) != -1:
            # delay for 10ms
            time.sleep(10 / 1000)
            if sound_play < 100:
                sound_play = sound_play + 1
                if sound_play == 100:
                    # save last face condition
                    last_played = condition
                    # catch no file error
                    try:
                        # call the function to play sound
                        play_sound(str(condition) + '.mp3')
                        # print last played file
                        print('Last played file: ', str(condition) + '.mp3')
                    except Exception as e:
                        # print error
                        print('Error: ' + str(e))
                    
                    
        # check if string does not exist
        if prev_detect.find(condition) == -1:
            # delay for 10ms
            time.sleep(10 / 1000)
            sound_play = 0
        # store previous detection
        prev_detect = condition
    
    # Show image back to screen
    cv2.imshow('Face Mode Detection', frame)
    # Breaking gracefully
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # stop sound
        if mixer.music.get_busy():
            mixer.music.pause()
            mixer.music.set_pos(0)
            mixer.music.stop()
        break
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step
Last played file:  happy.mp3
1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 16ms/step
Last played file:  neutral.mp3
1/1 [==============================] - 0s 16ms/step
